In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, datasets

# -------------------------------
# 1. Carregar e normalizar o MNIST
# -------------------------------
(x_train, _), (x_test, _) = datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# -------------------------------
# 2. Criar Autoencoder
# -------------------------------
latent_dim = 32  # tamanho do gargalo

# Encoder
inputs = tf.keras.Input(shape=(28,28,1))
x = layers.Flatten()(inputs)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
latent = layers.Dense(latent_dim, name="bottleneck")(x)
encoder = tf.keras.Model(inputs, latent, name="encoder")

# Decoder
latent_inputs = tf.keras.Input(shape=(latent_dim,))
x = layers.Dense(64, activation="relu")(latent_inputs)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(28*28, activation="sigmoid")(x)
outputs = layers.Reshape((28,28,1))(x)
decoder = tf.keras.Model(latent_inputs, outputs, name="decoder")

# Autoencoder
autoencoder = tf.keras.Model(inputs, decoder(encoder(inputs)), name="autoencoder")
autoencoder.compile(optimizer="adam", loss="mse")

# -------------------------------
# 3. Treinar Autoencoder
# -------------------------------
autoencoder.fit(x_train, x_train,
                epochs=200,
                batch_size=256,
                shuffle=True,
                # validation_data=(x_test, x_test),
                validation_split=0.2,  # 20% dos dados de treino usados para validação
                verbose=1)

# -------------------------------
# 4. Gerar novas imagens a partir do latent space
# -------------------------------
num_samples = 100  # quantas imagens gerar
latent_vectors = np.random.normal(size=(num_samples, latent_dim))  # amostra aleatória no espaço latente

generated_images = decoder.predict(latent_vectors)


In [ ]:
# -------------------------------
# 5. Visualizar imagens em várias linhas (5 imagens por linha)
# -------------------------------
images_per_row = 5
num_rows = int(np.ceil(num_samples / images_per_row))

plt.figure(figsize=(images_per_row*2, num_rows*2))
for i in range(num_samples):
    plt.subplot(num_rows, images_per_row, i+1)
    plt.imshow(generated_images[i].squeeze(), cmap='gray')
    plt.axis('off')
    # Se houver rótulo conhecido, exibe como título
    if 'labels_generated' in locals():
        plt.title(f"Digit: {labels_generated[i]}")
plt.suptitle("Novas imagens geradas pelo decoder")
plt.tight_layout()
plt.show()




plt.figure(figsize=(images_per_row*2, num_rows*2))
for i in range(num_samples):
    plt.subplot(num_rows, images_per_row, i+1)
    plt.imshow(generated_images[i].squeeze(), cmap='gray')
    plt.axis('off')
plt.suptitle("Novas imagens geradas pelo decoder")
plt.show()